### 모듈, 패키지 import

In [1]:
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta
import calendar
import matplotlib.pyplot as plt
from matplotlib.dates import MonthLocator, DateFormatter
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup
import urllib.parse as urlparse
import requests
from matplotlib import rc
rc('font', family = "Arial Unicode MS")
# from urllib3 import request, urlopen
import urllib.request

### 데이터 로드

In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/sample_submission.csv')

### train + test 붙이기 <- train, test 따로 전처리 하는 프로세스로 가고 싶으시면 이부분은 삭제하셔도 돼요!

In [3]:
train = train.append(test)
train

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0
...,...,...,...,...,...,...,...,...,...,...,...,...
45,2021-04-05,월,2973,125,174,704,331.0,모닝롤/커피콩빵 우유/주스 계란후라이 누룽지탕/흑미밥 청양콩나물국 스팸구이 양상추샐...,쌀밥/흑미밥/찰현미밥 쇠고기미역국 춘천닭갈비 오지치즈후라이 가지두반장볶음 포기김치 ...,흑미밥 돈육고추장찌개 갈치구이 김치전 취나물무침 깍두기,NaN,NaN
46,2021-04-06,화,2973,76,170,636,364.0,모닝롤/모닝샌드 우유/주스 계란후라이 고구마스프/흑미밥 아욱국 참치채소볶음 양상추샐...,쌀밥/귀리밥/찰현미밥 순두부백탕 매콤소갈비찜 깻잎완자전 돌나물초장무침 포기김치 시리...,추가밥 짬뽕*생면 수제찹쌀꿔바로우 메추리알곤약장조림 단무지무침 포기김치,NaN,NaN
47,2021-04-07,수,2973,96,214,1,334.0,모닝롤/호떡맥모닝 우유/주스 계란후라이 팥죽/흑미밥 닭살해장국 우엉채조림 양상추샐러...,쌀밥/흑미밥/찰현미밥 냉이국 돈육간장불고기 비빔냉면 오이나물볶음 겉절이김치 양상추샐...,단호박카레라이스 시금치된장국 소떡소떡 파프리카해초무침 감귤쥬스 포기김치,NaN,NaN
48,2021-04-08,목,2973,105,238,509,324.0,모닝롤/크로크무슈 우유/주스 계란후라이 누룽지탕/흑미밥 감자국 두부양념조림 양상추샐...,쌀밥/옥수수밥/찰현미밥 맑은떡국 (New)로제찜닭 가자미구이*장 유채나물무침 포기김...,흑미밥 어묵매운탕 쇠고기숙주볶음 채소계란찜 쑥갓생무침 김치볶음,NaN,NaN


### 공휴일 데이터 받기

In [4]:
# 정부 API 이용: 토, 일을 제외한 국경일 및 공휴일 추가
url = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService"
operation = "getRestDeInfo"
mykey = 'ac%2FSOW4KzOFKdu0z01iEVWGZd4TBl1MyiR04%2FfYmADthCjJBEyL73pewbapUk94Gm1%2FqtzMCban3C%2BpqU8c7ew%3D%3D'

date = []
datename = []
for year in range(2016, 2022):
    year = str(year)
    
    for month in range(1, 13):
        if month < 10:
            month = "0" + str(month)
        else:
            month = str(month)
            
        params = {'solYear' : year, 'solMonth' : month}
        rq_query = url +'/' + operation + '?' + urlparse.urlencode(params) + "&serviceKey=" + mykey    
        response = requests.get(rq_query) 
        dom = BeautifulSoup(response.content, "html.parser")
        
        items = dom.find_all("item")
        for item in items:
            date.append(item.locdate.string)
            datename.append(item.datename.string)

holiday_df= pd.DataFrame({"date": date, "datename": datename})

In [5]:
holiday_df.head()

,date,datename
0,20160101,신정
1,20160207,설날
2,20160208,설날
3,20160209,설날
4,20160210,대체공휴일


In [6]:
# 휴일 전날
holiday_before_df = holiday_df.copy()
holiday_before_df['date'] = holiday_before_df['date'].apply(lambda x : datetime.strptime(x,'%Y%m%d'))
holiday_before_df['date'] = holiday_before_df['date'].apply(lambda x : x - relativedelta(days=1))
holiday_before_df['date'] = holiday_before_df['date'].apply(lambda x : datetime.strftime(x, '%Y%m%d'))
holiday_before_df['datename'] = holiday_df['datename'].apply(lambda x : 1)
holiday_before_df.columns = ['일자', '휴일전날']
holiday_before_df.head()

,일자,휴일전날
0,20151231,1
1,20160206,1
2,20160207,1
3,20160208,1
4,20160209,1


### 이부분(휴일 다음날)도 최종 피쳐에 안쓰셨으면 삭제해주세요ㅎ

In [9]:
# 휴일 다음날
holiday_after_df = holiday_df.copy()
holiday_after_df['date'] = holiday_after_df['date'].apply(lambda x : datetime.strptime(x,'%Y%m%d'))
holiday_after_df['date'] = holiday_after_df['date'].apply(lambda x : x + relativedelta(days=1))
holiday_after_df['date'] = holiday_after_df['date'].apply(lambda x : datetime.strftime(x, '%Y%m%d'))
holiday_after_df['datename'] = holiday_df['datename'].apply(lambda x : 1)
holiday_after_df.columns = ['일자', '휴일다음날']
holiday_after_df.head()

,일자,휴일다음날
0,20160102,1
1,20160208,1
2,20160209,1
3,20160210,1
4,20160211,1


In [10]:
# 휴일전날 merge
train_df = pd.merge(train, holiday_before_df, left_on='일자', right_on='일자', how='left')
before_friday = train_df[train_df['요일']=='금'].index
train_df['휴일전날'][before_friday] = 1

In [11]:
# 휴일 다음날 merge
train_df = pd.merge(train_df, holiday_after_df, left_on='일자', right_on='일자', how='left')
after_sunday = train_df[train_df['요일']=='월'].index
train_df['휴일다음날'][after_sunday] = 1

In [12]:
# 평일은 0 처리
train_df['휴일전날'] = train_df['휴일전날'].fillna(0)
train_df['휴일다음날'] = train_df['휴일다음날'].fillna(0)

In [23]:
train_df.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,휴일전날,휴일다음날,점심비,저녁비
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,0.0,1.0,0.0,0.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,0.0,0.0,0.0,0.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,0.0,0.0,0.0,0.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0,0.0,0.0,0.0,0.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0,1.0,0.0,0.0,0.0


### 강수량 데이터 추가

In [16]:
# 기상청에서 다운로드한 진주시 강수량 데이터 활용
rain_df = pd.DataFrame()
for i in range(2016, 2022):
    tmp = pd.read_csv(f'./weather_data/{i}.csv', encoding='euc-kr')
    rain_df = rain_df.append(tmp)
rain_df.head()

,지점,지점명,일시,강수량(mm)
0,192,진주,2016-01-05 12:00,0.0
1,192,진주,2016-01-17 18:00,0.8
2,192,진주,2016-01-17 21:00,0.5
3,192,진주,2016-01-18 00:00,0.1
4,192,진주,2016-01-24 06:00,0.5


In [17]:
# 데이터 컬럼에 맞게 전처리
rain_df = rain_df[['일시', '강수량(mm)']].rename(columns={'강수량(mm)':'강수량'}).reset_index(drop=True)
rain_df['일시'] = rain_df['일시'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d %H:%M'))
rain_df['일자'] = rain_df['일시'].apply(lambda x : datetime.strftime(x, '%Y-%m-%d'))
rain_df['시간'] = rain_df['일시'].apply(lambda x : datetime.strftime(x, '%H'))
rain_df['시간'] = rain_df['시간'].astype('int')
rain_df = rain_df[['일자', '시간', '강수량']]
rain_df.head()

,일자,시간,강수량
0,2016-01-05,12,0.0
1,2016-01-17,18,0.8
2,2016-01-17,21,0.5
3,2016-01-18,0,0.1
4,2016-01-24,6,0.5


In [18]:
# 점심, 저녁 5미리 이상만 추출
lunch_rain_df = rain_df[(rain_df['시간'] == 12) & (rain_df['강수량'] >= 5)].reset_index(drop=True)
dinner_rain_df = rain_df[(rain_df['시간'] == 18) & (rain_df['강수량'] >= 5)].reset_index(drop=True)

In [19]:
lunch_rain_df.head()

,일자,시간,강수량
0,2016-02-12,12,9.4
1,2016-05-24,12,8.7
2,2016-06-24,12,10.8
3,2016-09-02,12,5.7
4,2016-09-17,12,10.2


In [20]:
# 점심비, 저녁비 merge
train_df = pd.merge(train_df, lunch_rain_df[['일자', '강수량']], left_on='일자', right_on='일자', how='left')
train_df['강수량'] = train_df['강수량'].fillna(0)
train_df = train_df.rename(columns={'강수량':'점심비'})

train_df = pd.merge(train_df, dinner_rain_df[['일자', '강수량']], left_on='일자', right_on='일자', how='left')
train_df['강수량'] = train_df['강수량'].fillna(0)
train_df = train_df.rename(columns={'강수량':'저녁비'})

In [22]:
train_df.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,휴일전날,휴일다음날,점심비,저녁비
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,0.0,1.0,0.0,0.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,0.0,0.0,0.0,0.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,0.0,0.0,0.0,0.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0,0.0,0.0,0.0,0.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0,1.0,0.0,0.0,0.0


### 원핫 인코딩(요일) <- 이부분은 순서 뒤로 미루셔도 됩니다 ㅎㅎ

In [46]:
onehot_tmp = pd.get_dummies(train_df['요일'])
train_df = pd.concat([train_df.drop(columns='요일'), onehot_tmp], axis=1)

In [24]:
train_df.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,휴일전날,휴일다음날,점심비,저녁비
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,0.0,1.0,0.0,0.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,0.0,0.0,0.0,0.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,0.0,0.0,0.0,0.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0,0.0,0.0,0.0,0.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0,1.0,0.0,0.0,0.0
